### RSI _Opposite_Order ( RSI_O2 ) Strategy

In [1]:
import time
from datetime import datetime

import pandas as pd
import pandas_ta as ta
import numpy as np

### import csv

In [2]:
data = pd.read_csv('EURUSD_15m_2022_01_03.csv')
data.tail()

,timestamp,open,high,low,close,adj close,volumn
3501,2021-12-31 20:45:00+00:00,1.138952,1.139212,1.138174,1.138174,1.138174,0
3502,2021-12-31 21:00:00+00:00,1.138045,1.138304,1.137915,1.138174,1.138174,0
3503,2021-12-31 21:30:00+00:00,1.138174,1.138174,1.138045,1.138045,1.138045,0
3504,2021-12-31 21:45:00+00:00,1.138174,1.138563,1.137010,1.137268,1.137268,0
3505,2021-12-31 22:00:00+00:00,1.137268,1.137268,1.137268,1.137268,1.137268,0


### create strategy

In [4]:
rsi_o2 = ta.Strategy(
    name = 'RSI 14 Days for opposite trade',
    description = 'RSI for alert',
    ta = [
        {'kind': 'ema', 'length': 200},
        {'kind': 'rsi', 'length': 24},
        {'kind': 'atr', 'length': 14}
    ]
)

In [5]:
df = data.copy()
df.ta.strategy(rsi_o2)
df.tail()

,timestamp,open,high,low,close,adj close,volumn,EMA_200,RSI_24,ATRr_14
3501,2021-12-31 20:45:00+00:00,1.138952,1.139212,1.138174,1.138174,1.138174,0,1.133923,59.300173,0.000759
3502,2021-12-31 21:00:00+00:00,1.138045,1.138304,1.137915,1.138174,1.138174,0,1.133965,59.300173,0.000733
3503,2021-12-31 21:30:00+00:00,1.138174,1.138174,1.138045,1.138045,1.138045,0,1.134006,58.433443,0.000690
3504,2021-12-31 21:45:00+00:00,1.138174,1.138563,1.137010,1.137268,1.137268,0,1.134038,53.540070,0.000751
3505,2021-12-31 22:00:00+00:00,1.137268,1.137268,1.137268,1.137268,1.137268,0,1.134070,53.540070,0.000698


### create signal

In [6]:
def backtest_rsi_o2(df, ema_len=200, rsi_len=14, atr_len=14, overbought=70, oversold=30, rr=1.5, pip_value=0.0001):
    
    rsi_o2 = ta.Strategy(
        name = 'RSI 14 Days for opposite trade',
        description = 'RSI for alert',
        ta = [
            {'kind': 'ema', 'length': ema_len},
            {'kind': 'rsi', 'length': rsi_len},
            {'kind': 'atr', 'length': atr_len}
        ]
    )
    
    df.ta.strategy(rsi_o2)
    df['action'] = np.nan

    orders = pd.DataFrame(columns = ['action', 'open time', 'open', 'close time', 'close', 'T/P', 'S/L','result'])
    current_action = 'close'
    order = {}
    unknow_result = []

    for index, row in df.iterrows():
        if current_action == 'close':
            
            if row['EMA_'+str(ema_len)] < row['low'] and row['RSI_'+str(rsi_len)] > overbought:
                row['action'] = 'buy'
                order['action'] = 'buy'
                order['open time'] = row['timestamp']
                order['open'] = row['open']
                order['S/L'] = row['low'] - row['ATRr_'+str(atr_len)]
                order['T/P'] = row['open'] + (rr * abs(row['open'] - order['S/L']))
                current_action = 'buy'
            
            elif row['EMA_'+str(ema_len)] > row['high'] and row['RSI_'+str(rsi_len)] < oversold:   
                row['action'] = 'sell'
                order['action'] = 'sell'
                order['open time'] = row['timestamp']
                order['open'] = row['open']
                order['S/L'] = row['high'] + row['ATRr_'+str(atr_len)]
                order['T/P'] = row['open'] - (rr * abs(row['open'] - order['S/L']))
                current_action = 'sell'
 
        if current_action == 'buy':
            if row['low'] <= order['S/L'] and row['high'] >= order['T/P']:
                unknow_result.append(row)
            if row['low'] <= order['S/L']:
                order['result'] = 'S/L'
                order['close time'] = row['timestamp']
                order['close'] = order['S/L']
                current_action = 'close'
                orders = orders.append(order, ignore_index = True)
                order = {}
            elif row['high'] >= order['T/P']:
                order['result'] = 'T/P'
                order['close time'] = row['timestamp']
                order['close'] = order['T/P']
                current_action = 'close'
                orders = orders.append(order, ignore_index = True)
                order = {}
                
        elif current_action == 'sell':
            if row['high'] >= order['S/L'] and row['low'] <= order['T/P']:
                unknow_result.append(row)
            if row['high'] >= order['S/L']:
                order['result'] = 'S/L'
                order['close time'] = row['timestamp']
                order['close'] = order['S/L']
                current_action = 'close'
                orders = orders.append(order, ignore_index = True)
                order = {}
            elif row['low'] <= order['T/P']:
                order['result'] = 'T/P'
                order['close time'] = row['timestamp']
                order['close'] = order['T/P']
                current_action = 'close'
                orders = orders.append(order, ignore_index = True)
                order = {}

    try:
        win_rate = round(len(orders[orders['result'] == 'T/P']) * 100 / len(orders), 2)
        loss_rate = round(len(orders[orders['result'] == 'S/L']) * 100 / len(orders), 2)
        result_info = f"total orders: {len(orders)}, overbought: {overbought}, oversold: {oversold} ,rr: {rr}\nema: {ema_len}, rsi: {rsi_len}, atr: {atr_len}\n"
        return {'win_rate': win_rate, 'info': result_info, 'unknow': unknow_result}
    except:
#         print("No orders to action")
        return {'win_rate': 0, 'info': 'No orders to action'}
   

In [10]:
orders = backtest_rsi_o2(df, ema_len=200, rsi_len=14, atr_len=14, overbought=70, oversold=30, rr=1.5)
# print(orders['win_rate'], '%,', orders['info'])
# 69.7 %, total orders: 165, overbought: 69, oversold: 31 ,rr: 1.5
# ema: 200, rsi: 14, atr: 14
print(f"{orders['win_rate']} %, {orders['info']}")

65.26 %, total orders: 95, overbought: 70, oversold: 30 ,rr: 1.5
ema: 200, rsi: 14, atr: 14



In [36]:
data_test = pd.read_csv('EURUSD_5m_2022_01_03.csv')
data_test.tail()

,timestamp,open,high,low,close,adj close,volumn
5757,2021-12-31 20:55:00+00:00,1.138433,1.138433,1.138174,1.138174,1.138174,0
5758,2021-12-31 21:05:00+00:00,1.138045,1.138045,1.137915,1.138045,1.138045,0
5759,2021-12-31 21:50:00+00:00,1.138304,1.138563,1.138304,1.138304,1.138304,0
5760,2021-12-31 21:55:00+00:00,1.138045,1.138045,1.137010,1.137268,1.137268,0
5761,2021-12-31 22:00:00+00:00,1.137268,1.137268,1.137268,1.137268,1.137268,0


In [46]:
# TODO prove that small timeframe can optimize win rate result.
# filter row from timeinterval 
# convert timestamp to index
# add column ['action', 'entry', 'S/L', 'T/P', 'result']

timestamp    2021-11-10 08:30:00+00:00
 open                          1.157274
 high                          1.157541
 low                           1.156872
 close                         1.156872
 adj close                     1.156872
 volumn                               0
 EMA_200                       1.158269
 RSI_24                       34.943941
 ATRr_14                       0.000615
 RSI_14                       29.902296
 action                            sell

'open time': '2021-11-10 08:30:00+00:00',
  'open': 1.157274,
  'S/L': 1.157486825405836,
  'T/P': 1.1569535892914535,
  'result': 'S/L',
  'close time': '2021-11-10 08:30:00+00:00',
  'close': 1.157486825405836},

# 2021-11-10 08:30:00+00:00	1.157274	1.157541	1.157274	1.157407	1.157407	0
# 2021-11-10 08:35:00+00:00	1.157274	1.157541	1.15714	1.15714	1.15714	0
# 2021-11-10 08:40:00+00:00	1.157274	1.157274	1.156872	1.156872	1.156872	0
# 2021-11-10 08:45:00+00:00	1.157006	1.157274	1.157006	1.15714	1.15714	0
data_test[data_test['timestamp'] == '2021-11-10 08:45:00+00:00']

,timestamp,open,high,low,close,adj close,volumn
680,2021-11-10 08:45:00+00:00,1.157006,1.157274,1.157006,1.15714,1.15714,0


In [51]:
1.1569535892914535 >= 1.15714 # T/P
# 1.157486825405836 <= 1.157541 # S/L

False

In [13]:
overB = 95
overS = 5
ema_list = [200] # list(range(100,210, 10))
rsi_list = list(range(10, 30+1))
atr_list = list(range(10, 30+1))
params_list = []

for ema in ema_list:
    for rsi in rsi_list:
        for atr in atr_list:
            for i in range(40):
                params_list.append([ema, rsi, atr, overB-i, overS+i, 1.5])
# params_list[5:10]
len(params_list)

17640

In [ ]:
good_params = {'win': 0, 'info': ''}
start = 2001
end = 5000

loop_num = start
for param in params_list[start:end]:
    result = backtest_rsi_o2(df, ema_len=param[0], rsi_len=param[1], atr_len=param[2], overbought=param[3], oversold=param[4], rr=param[5])
    if result['win_rate'] > good_params['win']:
        good_params['win'] = result['win_rate']
        good_params['info'] = result['info']
    print(f"run: {loop_num} to {end}", end = "\r")
    loop_num = loop_num + 1
    
print(f"\nwin rate: {good_params['win']} %\n{good_params['info']}")

In [12]:
# EURUSD 1 pip = 0.0001
# GBPUSD 1 pip = 0.0001
# USDJPY 1 pip = 0.01
# AUDUSD 1 pip = 0.0001
# 7.59 * 0.0001 == 0.000759
0.000759 / 0.01

0.0759

run: 1 to 1000
* total orders: 96, overbought: 76, oversold: 24 ,rr: 1.5
* ema: 200, rsi: 10, atr: 13
* win rate: 45.83 %

run: 1001 to 2000
* total orders: 110, overbought: 74, oversold: 26 ,rr: 1.5
* ema: 200, rsi: 11, atr: 14
* win rate: 45.45 %

### backtest

In [ ]:
'''
=== Todo ===
[/] import forex data from yohoo finance GBPUSD, EURUSD, USDJPY TF: 5min, 15min, 30min, 1hr, 4hr.
[/] backtest RSI_O2 strategy with forex.
[/] get data from binance BTCUSDT, ETHUSDT, BNBUSDT, ADAUSDT, SHIBUSDT, GALAUSDT, UNIUSDT, CAKEUSDT, SOLUSDT, LUNAUSDT, DOTUSDT
   TF: 15min, 30min, 1hr, 4hr, 1day
[] create EMA_3_lines strategy.
[] backtest EMA_3_lines strategy with forex.
[] create bot pull data from yahoo finance because they can give us last 2 months.
[] Get Free trial 30 days of Tradingview.
'''